In [1]:
import sys
sys.path.append('../../../../')

In [2]:
%load_ext autoreload
%autoreload 2

import torch
from jtvae import JTVAE

import jtnn
import fast_jtnn

path_adj = "./"

/data/bora.oztekin/anaconda3/envs/jtvae37f/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.
  return f(*args, **kwds)


In [3]:
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [4]:
from dig.ggraph.dataset import ZINC800
from dig.ggraph.evaluation import ConstPropOptEvaluator

In [5]:
dataset = ZINC800(one_shot=False, root=path_adj+'./datasets')

In [6]:
zinc_800_jt = torch.load(path_adj+"datasets/zinc_800_jt/processed/data.pt")

In [7]:
smiles = zinc_800_jt[-1]

In [8]:
# Define model
jtvae = JTVAE(smiles)

In [9]:
config = {"hidden_size": 420, "latent_size": 56, "depth": 3}
jtvae.get_model("cons_optim", config)

28 420


/data/bora.oztekin/anaconda3/envs/jtvae37f/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [10]:
property_values = zinc_800_jt[0]['y']

In [11]:
dataset = jtnn.PropDataset(zinc_800_jt[-1], property_values)

In [12]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4, collate_fn=lambda x:x, drop_last=True)

In [13]:
batch_size = 40
hidden_size = 200
latent_size = 56
depth = 3
beta = 1.0
lr = 1e-1 #1e-3
num_epochs = 1
jtvae.train_cons_optim(dataloader, batch_size, num_epochs, hidden_size, latent_size, depth, beta, lr)

/data/bora.oztekin/DIG/dig/ggraph/method/JTVAE/jtvae.py:213: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(param)
/data/bora.oztekin/DIG/dig/ggraph/method/JTVAE/jtvae.py:211: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(param, 0)
/data/bora.oztekin/anaconda3/envs/jtvae37f/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
  0%|          | 0/25 [00:00<?, ?it/s]

Model #Params: 4022K


 76%|███████▌  | 19/25 [01:43<00:30,  5.01s/it]

KL: 14.6, Word: 9.91, Topo: 57.27, Assm: 36.34, Steo: 41.95, Prop: 130.9817


100%|██████████| 25/25 [01:54<00:00,  4.57s/it]

learning rate: 0.072900



/data/bora.oztekin/anaconda3/envs/jtvae37f/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:449: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


In [ ]:
jtvae.run_cons_optim(smiles)